In [2]:
import pyrebase
import json
import pandas as pd

with open('../.streamlit/secrets.txt') as f:
    lines = f.read()
config = json.loads(lines)

firebase = pyrebase.initialize_app(config)

In [2]:
auth = firebase.auth()
db = firebase.database()
storage = firebase.storage()

In [ ]:
user = auth.create_user_with_email_and_password("sebastien.mauroo@patrick.com","123456")

In [67]:
try:
    signin = auth.sign_in_with_email_and_password("sebastien.mauroo@outlook.com","1234567")
except:
    signin ={"login failed"}

In [ ]:
signin


In [ ]:
firebase.auth().send_email_verification(signin['idToken'])

In [ ]:
firebase.auth().send_password_reset_email("sebastien.mauroo@outlook.com")

In [68]:
auth.get_account_info(signin['idToken'])

{'kind': 'identitytoolkit#GetAccountInfoResponse',
 'users': [{'localId': '8Ghnt52aWANjzWEUTwhWpFkp8832',
   'email': 'sebastien.mauroo@outlook.com',
   'passwordHash': 'UkVEQUNURUQ=',
   'emailVerified': True,
   'passwordUpdatedAt': 1677172127749,
   'providerUserInfo': [{'providerId': 'password',
     'federatedId': 'sebastien.mauroo@outlook.com',
     'email': 'sebastien.mauroo@outlook.com',
     'rawId': 'sebastien.mauroo@outlook.com'}],
   'validSince': '1677172127',
   'lastLoginAt': '1677964361227',
   'createdAt': '1677102536535',
   'lastRefreshAt': '2023-03-04T21:12:41.227Z'}]}

In [48]:

metermac_status = {"main_email_adress":["sebastien.mauroo@outlook.com"],"secundary_email":["chris@firmagalle.be","mauroogalle@telenet.be"]}
db.child("metermac").child("acbfbd98240").set(metermac_status)

{'main_email_adress': ['sebastien.mauroo@outlook.com'],
 'secundary_email': ['chris@firmagalle.be', 'mauroogalle@telenet.be']}

In [42]:
metermac_status = {"main_email_adress":["brecht.lauwers@outlook.com"]}
db.child("user").child("acbfbda4d6a").set(metermac_status)

{'main_email_adress': ['brecht.lauwers@outlook.com']}

In [55]:
metermac_status = {"main_email_adress":["brecht.lauwers@outlook.com"]}
db.child("test").child("sdf").set(metermac_status)

{'main_email_adress': ['brecht.lauwers@outlook.com']}

In [47]:
db.child("metermac").get().val()

OrderedDict([('acbfbd98240',
              {'main_email_adress': ['sebastien.mauroo@outlook.com'],
               'secundary_email': ['chris@firmagalle.be',
                'mauroogalle@telenet.be']}),
             ('acbfbda4d6a',
              {'main_email_adress': ['brecht.lauwers@outlook.com']})])

In [54]:
db.child("metermac").child("acbfbd@a4d6a").get().val()

In [63]:
all_metermac = list(db.child("metermac").shallow().get().val()) ##get all key values (shallow), and then make a readable list out of the val() you read



In [70]:
all_existing_meter_macs = list(db.child("metermac").shallow().get().val()) ##get all key values (shallow), and then make a readable list out of the val() you read
"acbfbd98240" in all_existing_meter_macs


True

In [65]:
'acbfbd98240' in all_metermac

True

In [3]:
dt = pd.read_csv("testdata_fluvius.csv",delimiter=';',decimal=',')

In [7]:
dt.head(5)

,Van Datum,Van Tijdstip,Tot Datum,Tot Tijdstip,EAN,Meter,Metertype,Register,Volume,Eenheid,Validatiestatus,start_time,end_time
0,14-10-2022,00:00:00,14-10-2022,00:15:00,"=""541448860008410850""",1SAG3100010760,Digitale Meter,Afname Nacht,NaN,kWh,Geen verbruik,2022-10-14 00:00:00,2022-10-14 00:15:00
1,14-10-2022,00:00:00,14-10-2022,00:15:00,"=""541448860008410850""",1SAG3100010760,Digitale Meter,Injectie Nacht,NaN,kWh,Geen verbruik,2022-10-14 00:00:00,2022-10-14 00:15:00
2,14-10-2022,00:15:00,14-10-2022,00:30:00,"=""541448860008410850""",1SAG3100010760,Digitale Meter,Afname Nacht,0.093,kWh,Gevalideerd,2022-10-14 00:15:00,2022-10-14 00:30:00
3,14-10-2022,00:15:00,14-10-2022,00:30:00,"=""541448860008410850""",1SAG3100010760,Digitale Meter,Injectie Nacht,0.000,kWh,Gevalideerd,2022-10-14 00:15:00,2022-10-14 00:30:00
4,14-10-2022,00:30:00,14-10-2022,00:45:00,"=""541448860008410850""",1SAG3100010760,Digitale Meter,Afname Nacht,0.120,kWh,Gevalideerd,2022-10-14 00:30:00,2022-10-14 00:45:00


In [5]:
###get start period
dt['start_time'] = pd.to_datetime(dt.iloc[:,0] + " " + dt.iloc[:,1],format='%d-%m-%Y %H:%M:%S')

###get end period
dt['end_time'] = pd.to_datetime(dt.iloc[:,2] + " " + dt.iloc[:,3],format='%d-%m-%Y %H:%M:%S')

In [6]:
#get repeated parameters
EAN_code = dt["EAN"].iloc[0].replace('=','').replace('"','')
Meter_code = dt["Meter"].iloc[0]
Meter_type = dt["Metertype"].iloc[0]
Power_unit = dt["Eenheid"].iloc[0]
Time_unit = (dt["end_time"].iloc[0] - dt["start_time"].iloc[0]).seconds/60 ##15 bij kwartier waarden, 60 bij uurwaarden
Data_period = (dt["start_time"].iloc[-1] - dt["start_time"].iloc[0]).round('d').days

In [12]:
Meter_code

'1SAG3100010760'

In [9]:
dt = dt.dropna()[['start_time','end_time','Volume','Register']]

In [ ]:
#########AANPASSEN NAAR INJECTIE

######injection analysis

#Estimated solar capacity
var1 = dt[dt['Register'].str.contains('Afname')]['Volume'].nlargest(3) ###largest 3 injectinos
Estimated_generation_capacity = var1.mean()*60/Time_unit ###60/Time_unit converts kWh towards kW

#Has solar panels
Has_solar_panels = True if Estimated_generation_capacity >= 0.6 else False  ###2 solar panels = +-600W

In [ ]:
dt[dt['Register'].str.contains('Afname')]['Volume'].nlargest(3) ###largest 3 injectinos

In [ ]:
dt

In [ ]:
graphtable = dt.pivot_table(index='end_time', columns='Register', values='Volume',aggfunc='mean').fillna(0)
graphtable['Afname'] = graphtable['Afname Dag'] + graphtable['Afname Nacht']
graphtable['Injectie'] = graphtable['Injectie Dag'] + graphtable['Injectie Nacht']
graphtable[['Afname','Injectie']]

In [ ]:
graphtable[['Afname','Injectie']]